In [ ]:
%pylab inline

In [ ]:
from plyfile import PlyData, PlyElement

In [ ]:
import glob

In [ ]:
def binner(data, binsize_cm, min_elevation):
    zmax = numpy.max(data['z'])
    xmin = numpy.min(data['x'])
    xmax = numpy.max(data['x'])
    ymin = numpy.min(data['y'])
    ymax = numpy.max(data['y'])
    
    xbinsize = (xmax-xmin)/binsize_cm
    ybinsize = (ymax-ymin)/binsize_cm
    
    data['x'] = mapper(data['x'], xmin, xmax)
    data['y'] = mapper(data['y'], ymin, ymax)

    hpoints = np.full((xbinsize,ybinsize), 0, dtype='float')

    xLen = xmax-xmin
    yLen = ymax-ymin

    for point in data:
        xapos = int(point[0]/xLen * xbinsize)-1
        yapos = int(point[1]/yLen * ybinsize)-1
        if hpoints[xapos][yapos] < point[2]:
            hpoints[xapos][yapos] = point[2]
    
    return hpoints

In [ ]:
@np.vectorize
def mapper(x, a, b):
    length = b-a
    return (x-a)/length * length

In [ ]:
in_files = glob.glob("/home/yanesl/Documents/exported_data_20160928_155653+0100/B/f00001_20*/*/*ply")
o_files = []
b_files = []
for f in in_files:
    if "sx001" not in f:
        if "boundary" not in f:
            o_files.append(f)
        else:
            b_files.append(f)

In [ ]:
for o, b in zip(o_files,b_files):
    boundarydata = PlyData.read(b)
    Odata        = PlyData.read(o)

    filename = ''.join(b.split("/")[-1].split("_")[0:3])
    
    odata = Odata['vertex'][:]
    bdata = boundarydata['vertex'][:]
    
    mean_height = mean(odata['z'])
    high = 2.*numpy.std(odata['z']) +mean_height
    mid = mean_height
    
    plt.figure(figsize=(10,20))
    ax1 = plt.subplot(1,3,1)
    plt.imshow(binner(odata, 1.3, high).transpose(), interpolation='gaussian')
    plt.colorbar()
    ax2 = plt.subplot(1,3,2,sharex=ax1, sharey=ax1)
    plt.imshow(binner(odata[ (odata['z'] > 600) & (odata['z'] < 800) ], 1.3, high).transpose(), interpolation='gaussian')
    plt.colorbar()
    ax3 = plt.subplot(1,3,3, sharex=ax1, sharey=ax1)
    plt.imshow(binner(odata[ (odata['z'] < 10) | (odata['z'] > 800) ], 1.3, high).transpose(), interpolation='gaussian')
    plt.colorbar()
    plt.savefig(filename+".png", format="png")
    plt.close()

In [ ]:
plt.figure(figsize=(10,10))
plt.hist(bdata['z'][bdata['z']>50].flatten(), bins=70);

In [ ]:
print mean_height
print high
print mid

print numpy.max(odata['y'])
print numpy.max(odata['x'])

In [ ]:
# plt.xlim(0,760)
# plt.ylim(0,6000)
